## Challenge code
 
 On one hand you are asked to complete `circuit` (you only need to apply gates).
 On the other hand you must complete `process_output`, which will take the output of `circuit` (a vector of dimension two where each term can take the value 0 or 1) and will return 2 if $U = RY(2\pi/3)$ or 4 if $U = RY(4\pi/3).$
 
 ![img](images/2is4.jpeg)
 
 ### Output
 
 The circuit function will receive the gate $U$ that you are asked to determine. To judge whether your circuit works as expected, we will randomly send 5000 different examples and check that they always classified correctly. Therefore, in this question there are no public or private test cases. They are randomly generated.
 Good luck!
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [12]:
import json
import pennylane as qml
from pennylane import numpy as np
import sympy as sp

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [13]:
# with this circuit we have a one to one correspondence between the
# U gate chosen and the output

dev = qml.device("default.qubit", wires=2, shots=1)
@qml.qnode(dev)
def guess_gate(theta):
    qml.Hadamard(wires=0)
    qml.RY(theta, wires=1)
    qml.CRY(theta, wires=[0, 1])
    qml.Hadamard(wires=0)
    qml.Barrier(only_visual=True)
    qml.CNOT(wires=[0, 1])
    qml.RY(-np.pi/3, wires=0)
    qml.ctrl(qml.PauliX, control=[1], control_values=[0])(wires=[0])
    return qml.sample(wires=[1])

theta_1 = 2*np.pi/3
print(qml.draw(guess_gate)(theta_1), '\n')
print(np.round(guess_gate(theta_1), 3), '\n\n')

theta_2 = 4*np.pi/3
print(qml.draw(guess_gate)(theta_2), '\n')
print(np.round(guess_gate(theta_2), 3))

0: ──H────────╭●─────────H──||─╭●──RY(-1.05)─╭X─┤        
1: ──RY(2.09)─╰RY(2.09)─────||─╰X────────────╰○─┤  Sample 

1 


0: ──H────────╭●─────────H──||─╭●──RY(-1.05)─╭X─┤        
1: ──RY(4.19)─╰RY(4.19)─────||─╰X────────────╰○─┤  Sample 

0


In [15]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def qn(theta):
    qml.RY(theta, wires=0)
    qml.RY(theta, wires=0)
    qml.RY(theta, wires=0)
    return qml.state()

theta_1 = 2*np.pi/3
print(qml.draw(qn)(theta_1), '\n')
RY_2x3 = qml.matrix(qn)(theta_1)
display(sp.Matrix(np.round(RY_2x3, 3)))
print('\n\n')

theta_2 = 4*np.pi/3
print(qml.draw(qn)(theta_2), '\n')
RY_4x3 = qml.matrix(qn)(theta_2)
display(sp.Matrix(np.round(RY_4x3, 3)))

0: ──RY(2.09)──RY(2.09)──RY(2.09)─┤  State 



Matrix([
[-1.0,    0],
[   0, -1.0]])




0: ──RY(4.19)──RY(4.19)──RY(4.19)─┤  State 



Matrix([
[1.0,   0],
[  0, 1.0]])

Applying 3 times the RY gate with angles 2 * pi/2 or 4 * pi/2 is equal to minus the identity or the identity (respectively)

In [ ]:
dev = qml.device("default.qubit", wires=2, shots=1)

@qml.qnode(dev)
def circuit(theta):
    qml.Hadamard(wires=0)
    qml.CRY(theta, wires=[0, 1])
    qml.CRY(theta, wires=[0, 1])
    qml.CRY(theta, wires=[0, 1])
    qml.Hadamard(wires=0)
    return qml.sample(wires=[0])

theta_1 = 2*np.pi/3
print(qml.draw(circuit)(theta_1), '\n')
print(np.round(circuit(theta_1), 3), '\n\n')

theta_2 = 4*np.pi/3
print(qml.draw(circuit)(theta_2), '\n')
print(np.round(circuit(theta_2), 3), '\n\n')

0: ──H─╭●────────╭●────────╭●─────────H─┤  Sample
1: ────╰RY(2.09)─╰RY(2.09)─╰RY(2.09)────┤         

1 


0: ──H─╭●────────╭●────────╭●─────────H─┤  Sample
1: ────╰RY(4.19)─╰RY(4.19)─╰RY(4.19)────┤         

0 




---

In [33]:
# Uneditable section #
dev = qml.device("default.qubit", wires=2, shots=1)
# dev.operations.add("op")
# dev.operations.add("C(op)")
# End of uneditable section #

@qml.qnode(dev)
def circuit(U):
    """This will be the circuit you will use to determine which of the two angles we have.
    Remember that only a single shot will be executed.

    Args:
        U (qml.ops): It is the gate to discriminate between  RY(2pi/3) or RY(4pi/3).

    Returns:
        (numpy.tensor): Vector of two elements representing the output measured in each of the qubits.
    """
    # Put your code here #
    # to use U,  call 'U(wires = <wire where you want to apply the gate>)'
    # to use Control-U, call 'qml.ctrl(U, control = <control wire>)(wires = <wire where you want to apply the gate>)'
    qml.Hadamard(wires=0)
    qml.ctrl(U, control=0)(wires=1)
    qml.ctrl(U, control=0)(wires=1)
    qml.ctrl(U, control=0)(wires=1)
    qml.Hadamard(wires=0)
    return qml.sample(wires=range(2))

def process_output(measurement):
    """This function processes the output of the circuit to discriminate between gates.

    Args:
        measurement (numpy.array): Output of the previous circuit a vector of dimension 2.

    Returns:
        (int): return 2 or 4 depending on the associated RY gate.
    """
    # Put your code here #
    top_qubit = measurement[0]
    if top_qubit:
        return 2
    else:
        return 4

These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [34]:
def run(test_case_input: str) -> str:
    return None

def check(solution_output: str, expected_output: str) -> None:
    numbers = 2 * np.random.randint(1, 3, 5000)

    def U2(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(2 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    def U4(wires):
        class op(qml.operation.Operator):
            num_wires = 1

            def compute_decomposition(self, wires):
                raise ValueError("You cannot decompose this gate")

            def matrix(self):
                return qml.matrix(qml.RY(4 * np.pi / 3, wires=3))

        op(wires=wires)
        return None

    output = []
    for i in numbers:
        if i == 2:
            U = U2
        else:
            U = U4
        out = circuit(U)
        output.append(process_output(out))

    assert np.allclose(
        output, numbers, rtol=1e-4
    ), "Your circuit does not give the correct output."

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: No input
 	+ expected output: No output
 - input: No input
 	+ expected output: No output

In [35]:
test_cases = [['No input', 'No output'], ['No input', 'No output']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [36]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...
Correct!
Running test case 1 with input 'No input'...
Correct!
